In [1]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('Reviews.csv')
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [3]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [4]:
# Create stopword list:
stopwords = set(STOPWORDS)
stopwords.update(["br", "href"])

In [5]:
textt = " ".join(review for review in df.Text)
#wordcloud = WordCloud(stopwords=stopwords).generate(textt)
#plt.imshow(wordcloud, interpolation='bilinear')
#plt.axis("off")
#plt.savefig('wordcloud11.png')
#plt.show()

In [6]:
# assign reviews with score > 3 as positive sentiment
# score < 3 negative sentiment
# remove score = 3
df = df[df['Score'] != 3]
df['sentiment'] = df['Score'].apply(lambda rating : +1 if rating > 3 else -1)

In [7]:
##Data Cleaning

In [8]:
def remove_punctuation(text):
    final = "".join(u for u in text if u not in ("?", ".", ";", ":",  "!",'"'))
    return final
df['Text'] = df['Text'].apply(remove_punctuation)
df = df.dropna(subset=['Summary'])
df['Summary'] = df['Summary'].apply(remove_punctuation)

In [9]:
dfNew = df[['Summary','sentiment']]
dfNew.head()

,Summary,sentiment
0,Good Quality Dog Food,1
1,Not as Advertised,-1
2,Delight says it all,1
3,Cough Medicine,-1
4,Great taffy,1


In [117]:
X = dfNew['Summary']
y = dfNew['sentiment']

In [116]:
#import spacy
#!python -m spacy download en_core_web_sm

You should consider upgrading via the 'C:\Users\queenie.d.pamatian\Anaconda3\python.exe -m pip install --upgrade pip' command.



[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [195]:
from sklearn import model_selection, linear_model, metrics
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, random_state=0)

In [154]:
#docs = [nlp(text) for text in X_train]
#train_x_word_vectors = [x.vector for x in docs]

In [196]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
V_X_train = vectorizer.fit_transform(X_train)
V_X_test = vectorizer.transform(X_test)

In [197]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [198]:
lr.fit(V_X_train,y_train)

C:\Users\queenie.d.pamatian\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [199]:
predictions = lr.predict(V_X_test)


In [200]:
import numpy as np

In [201]:
# find accuracy, precision, recall:
from sklearn.metrics import confusion_matrix,classification_report
new = np.asarray(y_test)
confusion_matrix(predictions,y_test)

array([[ 13580,   2878],
       [  7010, 107980]], dtype=int64)

In [202]:
print(classification_report(predictions,y_test))

              precision    recall  f1-score   support

          -1       0.66      0.83      0.73     16458
           1       0.97      0.94      0.96    114990

    accuracy                           0.92    131448
   macro avg       0.82      0.88      0.84    131448
weighted avg       0.93      0.92      0.93    131448



In [203]:
#key model metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
print("accuracy: ",accuracy_score(y_test, predictions))
print("confusion matrix: ",confusion_matrix(y_test, predictions))
print("precision: ",precision_score(y_test,predictions))

accuracy:  0.9247763374109914
confusion matrix:  [[ 13580   7010]
 [  2878 107980]]
precision:  0.9390381772328028


In [210]:
narrative = vectorizer.inverse_transform(V_X_test)

In [211]:
results = pd.DataFrame()
results['narrative'] = narrative
results['rating'] = predictions

In [242]:
new = ['this product sucks','this is a hot product','mediocre','kinda cool','below average','dope','disgusting', 'i am unhappy']
new = vectorizer.transform(new)

In [243]:
lr.predict(new)

array([-1,  1, -1,  1, -1,  1, -1, -1], dtype=int64)

In [241]:
results

,narrative,rating
0,"[best, corn, for, oil, popping, the]",1
1,[perfect],1
2,[heavenly],1
3,"[guys, little, perfect]",1
4,"[broken, package]",-1
...,...,...
131443,"[favorite, jerky, my]",1
131444,"[favorite, my]",1
131445,"[eh, ok]",-1
131446,"[seems, to, work]",1
